In [1]:
import pandas as pd
import random
from datetime import timedelta
from datetime import datetime

# This is a sample of what a product sheet would look like
# ID for vendor will mean something such as type of product or payment type
product_df = pd.DataFrame({
    "ProductID": [1,2,3,4],
    "Product": ["eggs","wine","cheese","candles"],
    "Vendor": [3001, 3002, 3003, 3004],
    "Unit_Price": [3.20, 16.20, 5.20,20.5],
    "Unit_Cost": [1.20, 3.40, 2.10, 10],
    "Initial_Inventory": [0,0,0,0],
    "Spread":[.30, .50, .10, .10],
    "PerInvoiceRng_Min":[10,10,10,10],
    "PerInvoiceRng_Max":[20,20,20,20]
})

# This would all come in as JSON not created in Python
Initial_Year = 2010
YearsIn_Operation = 10
first_year_rev = 10000000
trend = .05
modjulation = .01

x = first_year_rev
y = Initial_Year
coin = (1,2)

list_years = []



for year in range(YearsIn_Operation):
  Revenue = x
  Year = y
  Trended = Revenue * (1+trend)
  direction = random.sample(coin,1)
  if direction == 1:
    Modjulated = Trended * (1+modjulation)
  else:
    Modjulated = Trended * (1-modjulation)
  list_year_item = [Year, round(Revenue,2)]
  list_years.append(list_year_item)
  x = Modjulated
  y = y+1

# This is what the Data would look like in JSON
rev_df = pd.DataFrame(list_years,columns=["Year","Rev"])


In [122]:
from re import S
class PreProcessing:
  def __init__(self, product_df, year, Year_Rev):
    self.revenue = Year_Rev[1]
    self.products = product_df
    self.year = year

  def ProductProcessing(self):
    self.product_id_list = self.products['ProductID']
    self.vendor_list = self.products['Vendor']
    self.initial_inventory_list = self.products['Initial_Inventory']

    self.product_cost_info = self.products[['Product', 'Spread',
                                            'Unit_Price', 'Unit_Cost',
                                            'PerInvoiceRng_Min',
                                            'PerInvoiceRng_Max']].sort_values(by='Spread', ascending=False)
    product_cost = self.product_cost_info
    Rev = self.revenue
    yr_Product_Rev = []
    yr_Quantity_Sold = []
    Product_Cost = []
    Selling_Price = []
    yr_Product = []
    yr_noRecords_min = []
    yr_noRecords_max = []
    for product in self.product_cost_info['Product']:
      noRecords_min = product_cost[product_cost['Product']==product]['PerInvoiceRng_Min']
      noRecords_max = product_cost[product_cost['Product']==product]['PerInvoiceRng_Max']
      spread = product_cost[product_cost['Product']==product]['Spread'].item()
      unit_price = product_cost[product_cost['Product']==product]['Unit_Price'].item()
      unit_cost = product_cost[product_cost['Product']==product]['Unit_Cost'].item()
      price_sold = self.revenue * spread
      q_sold = round(price_sold / unit_price,0)
      p_sold = q_sold*unit_price
      if Rev >= p_sold:
        Rev = Rev - p_sold
      else:
        q_sold = round(Rev / unit_price,0)
        p_sold = q_sold*unit_price

      yr_Product.append(product)
      yr_Product_Rev.append(p_sold)
      yr_Quantity_Sold.append(q_sold)
      Product_Cost.append(unit_cost)
      Selling_Price.append(unit_price)
      yr_noRecords_min.append(noRecords_min)
      yr_noRecords_max.append(noRecords_max)

    processed_product_lists = [yr_Product, yr_Product_Rev,
                               yr_Quantity_Sold, Product_Cost, Selling_Price,
                               yr_noRecords_min, yr_noRecords_max]

    self.Processed_Product_df = pd.DataFrame(processed_product_lists).T
    self.Processed_Product_df.columns = ["yr_Product","yr_Product_Rev",
                                         "yr_Quantity_Sold","Product_Cost","Selling_Price",
                                         "yr_noRecordsMin","yr_noRecordsMax"]
    return [self.year, self.Processed_Product_df]


# Generate Sales Invoices
class CompanyGeneration:
  def __init__(self, yera, product, yr_product_data):
    self.year = yera
    self.yr_product_data = yr_product_data

  def GenerateSales(self): # Already Per Product
    print(self.year)
    yr_Quantity_Sold = yr_product_data[yr_product_data['yr_Product']==product]['yr_Quantity_Sold'].item()
    Product_Cost = yr_product_data[yr_product_data['yr_Product']==product]['Product_Cost'].item()
    Selling_Price = yr_product_data[yr_product_data['yr_Product']==product]['Selling_Price'].item()
    MinRecords_PerInvoice = int(yr_product_data[yr_product_data['yr_Product']==product]['yr_noRecordsMin'])
    MaxRecords_PerInvoice = int(yr_product_data[yr_product_data['yr_Product']==product]['yr_noRecordsMax'])

    # Initial Quantity
    Quantity_Remaining = yr_Quantity_Sold
    Yr_Start = datetime(self.year, 1, 1)
    Yr_End = datetime(self.year, 12, 31)

    Sales_df_rows = []
    x = 0

    while Quantity_Remaining > 0:
      Order_Date = Yr_Start + timedelta(days=random.randint(0, (Yr_End-Yr_Start).days))
      # Product is Shipped 1-30 days after order
      Shipment_Date = Order_Date + timedelta(days = random.randint(1, 30))
      Records_PerInvoice = random.randint(MinRecords_PerInvoice, MaxRecords_PerInvoice)
      if Records_PerInvoice > Quantity_Remaining:
        Records_PerInvoice = Quantity_Remaining
      else:
        pass
      Rev = Records_PerInvoice * Selling_Price
      Tcost = Records_PerInvoice * Product_Cost
      Sales_df_rows.append([product, Records_PerInvoice, Order_Date, Shipment_Date, Selling_Price, Product_Cost, Rev, Tcost])
      Quantity_Remaining = Quantity_Remaining - Records_PerInvoice

      # Record Journal Entries for Rev and Cash or AR

    Sales_df = pd.DataFrame(Sales_df_rows, columns = ["Product", "Quantity",
                                                      "Order_Date", "Shipment_Date",
                                                      "Selling_Price", "Product_Cost",
                                                      "Rev", "Total_cost"])

    cleaned_df = Sales_df[['Shipment_Date', 'Rev']].copy()
    cleaned_df.rename(columns={'Shipment_Date': 'Date', 'Rev': 'Amount'}, inplace=True)
    DrAcc = random.choices([1000,1010],weights=[70,30],k=len(Sales_df))
    CrAcc = 4000
    RecordingFunctions().JournalEntries_Bulk(cleaned_df,1000,4000,"dr",1010,1000,70,30)

    return [self.year, Sales_df]



# Helper Functions
class RecordingFunctions:
  def __init__(self):
    pass

  # Bulk Data Processing
  def JournalEntries_Bulk(self, Clean_df, DebitAcc, CreditAcc, probability=None, val1=0, val2=0,p1=0,p2=0 ):
    if probability == None:
      pass
    elif probability.lower() == "dr":
      DebitAcc = random.choices([val1,val2],weights=[p1,p2],k=len(Clean_df))
    elif probability.lower() == "cr":
      CreditAcc = random.choices([val1,val2],weights=[p1,p2],k=len(Clean_df))

    df_debits = Clean_df[['Date','Amount']].copy()
    df_debits['Account'] = DebitAcc
    df_debits['Balance'] = "Debit"

    # print(df_debits.head())
    Data_manager().ConcatData('journal',df_debits)

    df_credits = Clean_df[['Date','Amount']].copy()
    df_credits['Account'] = CreditAcc
    df_credits['Balance'] = "Credit"

    # print(df_credits.head())
    Data_manager().ConcatData('journal',df_credits)



  # Row by Row Data Processing
  def JournalEntries_Single(self, Date, Amount, DebitAcc, CreditAcc):
    self.General_Journal = pd.DataFrame({"Date":[],"Amount":[],"Account":[],"Balance":[]})

    if isinstance(Amount, list) and isinstance(DebitAcc, list) and isinstance(CreditAcc, list):
      if len(Amount) == len(DebitAcc) + len(CreditAcc):
        len_ofDr = len(DebitAcc)
        DrAmounts = Amount[:len_ofDr]
        CrAmounts = Amount[len_ofDr:]
        Debits = zip(DebitAcc,DrAmounts)
        Credits = zip(CreditAcc,CrAmounts)
        for dr, amount in Debits:
          Journal_Entry = [Date, dr, amount, "Debit"]
          self.General_Journal.loc[len(self.General_Journal)] = Journal_Entry
        for cr, amount in Credits:
          Journal_Entry = [Date, cr, amount, "Credit"]
          self.General_Journal.loc[len(self.General_Journal)] = Journal_Entry
      else:
        print("ERROR Creating Journal Entries - Incorrect Number of Amounts")

    elif isinstance(Amount, int) or isinstance(Amount, float):
        Journal_Entry = [Date,DebitAcc, Amount, "Debit"]
        self.General_Journal.loc[len(self.General_Journal)] = Journal_Entry

        Journal_Entry = [Date, CreditAcc, Amount, "Credit"]
        self.General_Journal.loc[len(self.General_Journal)] = Journal_Entry

    else:
      print("ERROR Creating Journal Entry - Incorrect DataTypes")

    Data_manager().ConcatData('journal',self.General_Journal)



class Data_manager:
  def __init__(self):
    self.MasterGeneralJournal = pd.DataFrame(columns = ["Date","Account","Amount","Balance"])
    # Add the Rest of the Finacial Statements Here

  def RequestData(self,file):
    if 'journal' in file.lower():
      return self.MasterGeneralJournal
    elif 'income' in file.lower():
      pass
    elif 'balance' in file.lower():
      pass

  def ConcatData(self,file,new_data):
    if 'journal' in file.lower():
      self.MasterGeneralJournal = pd.concat([new_data, self.MasterGeneralJournal],ignore_index=True)
    elif 'income' in file.lower():
      pass
    elif 'balance' in file.lower():
      pass

  def PrintData(self,file):
    pass



In [ ]:
Yr_Product_dict = {}
General_Journal = pd.DataFrame(columns = ["Date","Account","Amount","Balance"])


for yr in rev_df["Year"]:
  # Processing Products
  Year_Rev = rev_df[rev_df['Year']==yr].values.tolist()[0]
  generator_instance = PreProcessing(product_df, yr, Year_Rev)
  yr_product_data = generator_instance.ProductProcessing()[1]
  for product in yr_product_data['yr_Product'].unique():
    CompanyGeneration(yr, product, yr_product_data).GenerateSales()
    break

  break


In [ ]:
# Testing New Journal Entry Data

cleandf = pd.DataFrame({"Date":[1,2,3,4,5,6,7,8,9,10],"Amount":[1,2,3,4,5,6,7,8,9,10]})

# RecordingFunctions().JournalEntries_Bulk(cleandf,1000,4000,"dr",1000,2000,40,60)

RecordingFunctions().JournalEntries_Single("1/1/2023", 100, 1000, 2000)

